# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
import json
import subprocess
import numpy as np
from statistics import mean
from collections import Counter
from rank_bm25 import BM25Okapi

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [2]:
# Load the datasets
with open('data/train-claims.json', 'r') as input_file:
    train_claim_data = json.load(input_file)

with open('data/dev-claims.json', 'r') as input_file:
    dev_claim_data = json.load(input_file)

with open('data/test-claims-unlabelled.json', 'r') as input_file:
    test_claim_data = json.load(input_file)

with open('data/evidence.json', 'r') as input_file:
    evi_data = json.load(input_file)

In [3]:
# Prepare data for vectorization
print("Prepare data for vectorization")
full_evidence_id = list(evi_data.keys())
full_evidence_text = list(evi_data.values())
train_claim_id = list(train_claim_data.keys())
train_claim_text = [v["claim_text"] for v in train_claim_data.values()]

# Tokenize the text for BM25
print("Tokenize the text for BM25")
tokenized_evidence = [doc.split() for doc in full_evidence_text]
tokenized_train_claims = [doc.split() for doc in train_claim_text]

# Initialize BM25
print("Initialize BM25")
bm25_evidence = BM25Okapi(tokenized_evidence)
bm25_claims = BM25Okapi(tokenized_train_claims)

Prepare data for vectorization
Tokenize the text for BM25
Initialize BM25


In [ ]:
# Prepare output dictionary
print("Prepare output dictionary")
output_results = {}

# Iterate over each claim
print("Iterate over each claim")
for claim_id, claim_value in dev_claim_data.items():
    # Tokenize the current claim text
    tokenized_claim = claim_value['claim_text'].split()

    # Retrieve top-k evidence using BM25
    evidence_scores = bm25_evidence.get_scores(tokenized_claim)
    # Get top 1/2/3/4/5/6/7/8/9/10
    top_k_evidence_indices = np.argsort(-evidence_scores)[:3]
    selected_evidence_ids = [full_evidence_id[idx] for idx in top_k_evidence_indices]
    output_results[claim_id] = {"evidences": selected_evidence_ids}

    # Retrieve the most similar claim from the training set using BM25
    claim_scores = bm25_claims.get_scores(tokenized_claim)
    most_similar_claim_idx = np.argmax(claim_scores)
    output_results[claim_id]["claim_label"] = train_claim_data[train_claim_id[most_similar_claim_idx]]["claim_label"]

# Write results to file
print("Write results to file")
with open("data/dev_predict.json", "w") as outfile:
    json.dump(output_results, outfile, indent=4)

Prepare output dictionary
Iterate over each claim


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
# %%cmd
# python eval.py --predictions dev-claims-baseline.json --groundtruth dev-claims.json
# python eval.py --predictions dev_predict.json --groundtruth dev-claims.json

output = subprocess.check_output("python eval.py --predictions data/dev_predict.json --groundtruth data/dev-claims.json", shell=True)

# Print
print(output.decode("utf-8"))

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*